In [1]:
import config
import event
import handler
import strategy
import portfolio
import risk
import analysis as an
from datetime import datetime

#SQL & API credentials. Store in config file.

access_token = config.access_token
account_id = config.account_id

sql_host = config.sql_host
sql_user = config.sql_user_analysis
sql_password = config.sql_password_analysis

In [ ]:
for i in range(0, df.shape[0]):
    if df.iloc[i]['State'] != df.iloc[i-1]['State']:
        three_hour_trend = an.state(an.selectlast('H3', df['time'].iloc[i], 10))
        if df.iloc[i]['State'] == three_hour_trend:
            print(df['time'].iloc[i], df.iloc[i]['State'], an.state(an.selectlast('H3', df['time'].iloc[i], 10))) 

In [2]:
dh = handler.DataHandler()
queue = event.event_queue()
strategy = strategy.Strategy()

In [3]:
queue.add_to_queue(dh.get_latest_rate())

In [4]:
queue.handle_next()

NameError: name 'strategy' is not defined

In [ ]:
queue.queue[0]

In [ ]:
ev = strategy.generate_signal(queue.queue[0])

In [ ]:
ev.signal